In [1]:
# Coronary-Artery-Tracking-via-3D-CNN-Classification

In [2]:
pwd

'C:\\Users\\dhkim\\Desktop\\directory\\k-ium'

In [3]:
import numpy as np
import pandas as pd
import math
import random
import os
import shutil
import torch
import glob
from tqdm import tqdm
from source import model
# from source import dataset
from source import utils
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from custom_dataset_3d import Custom_dataset

In [4]:
pwd

'C:\\Users\\dhkim\\Desktop\\directory\\k-ium'

In [5]:
train_csv = pd.read_csv('data/2023_k_ium_composition/train_set/train.csv')

In [6]:
train_csv

,Index,Aneurysm,L_ICA,R_ICA,L_PCOM,R_PCOM,L_AntChor,R_AntChor,L_ACA,R_ACA,...,R_MCA,L_VA,R_VA,L_PICA,R_PICA,L_SCA,R_SCA,BA,L_PCA,R_PCA
0,1001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1004,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1005,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
4,1006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2607,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1123,2608,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1124,2609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1125,2610,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_csv[train_csv['Index'] == 1001]

,Index,Aneurysm,L_ICA,R_ICA,L_PCOM,R_PCOM,L_AntChor,R_AntChor,L_ACA,R_ACA,...,R_MCA,L_VA,R_VA,L_PICA,R_PICA,L_SCA,R_SCA,BA,L_PCA,R_PCA
0,1001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 3d train, val 5:2로 폴더나누기

In [8]:
pwd

'C:\\Users\\dhkim\\Desktop\\directory\\k-ium'

In [9]:
patient_3d_img_list = glob.glob('data\\3d_patient_list/*/*')
patient_3d_img_list

os.makedirs(f"C:/Users/dhkim/Desktop/directory/k-ium/data/train_3d", exist_ok=True)
os.makedirs(f"C:/Users/dhkim/Desktop/directory/k-ium/data/valid_3d", exist_ok=True)



In [10]:
len(patient_3d_img_list)

1127

In [11]:
pwd

'C:\\Users\\dhkim\\Desktop\\directory\\k-ium'

In [12]:
# # patient_list = os.listdir("patient_list")
# # patient_list

# # 비율 계산
# total_patients = len(patient_3d_img_list)
# train_ratio = 5 / 7
# val_ratio = 2 / 7

# # # 환자를 train, val, test 폴더로 복사
# train_patients = patient_3d_img_list[:int(total_patients * train_ratio)]
# val_patients = patient_3d_img_list[int(total_patients * train_ratio):]

# # print(len(train_patients))

# for patient_3d in train_patients:
# #     print(patient_3d)
#     source_path=patient_3d
# #     print(source_path)
#     destination_path = "C:/Users/dhkim/Desktop/directory/k-ium/data/train_3d"
#     shutil.copy(source_path, destination_path)
    

# for patient_3d in val_patients:
#     source_path = patient_3d
#     destination_path = "C:/Users/dhkim/Desktop/directory/k-ium/data/valid_3d"
#     shutil.copy(source_path, destination_path)



In [13]:
a = glob.glob('data/train_3d/*')
len(a)

805

In [14]:
a = glob.glob('data/valid_3d/*')
len(a)

322

# 모델링

In [15]:
random.seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [16]:
# from torch.utils.data import Dataset, DataLoader
# import cv2
# import os
# import glob
# import torch
# import utils 

# class Custom_dataset(Dataset):
#     def __init__(self, root_path, mode, transforms):
#         self.all_data = sorted(glob.glob(os.path.join(root_path, mode, '*')))
#         self.transforms = transforms
# #         print(self.all_data)
        
#     def __preproc__(self, file):
#         verts, faces = utils.read_off(file)
#         if self.transforms:
#             pointcloud = self.transforms((verts, faces))
#         return pointcloud
        
#     def __getitem__(self, index):
#         images = []
#         if torch.is_tensor(index):
#             index = index.tolist()
        
#         pcd_path = self.all_data[index]
# #         print(pcd_path)
        
#         with open(pcd_path, 'r') as f:
#             pointcloud = self.__preproc__(f)
# #         print("~~")
#         pat_number = pcd_path[53:57]
# #         print(pat_number)

#         patient_index=train_csv[train_csv['Index'] == int(pat_number)]
# #         print(patient_index)
#         patient_aneurysm = patient_index['Aneurysm']
# #         print(patient_aneurysm)
#         label = int(patient_aneurysm.values)
        
        
# #         print(pointcloud)
# #         print(label)
#         return pointcloud, label       
            
#     def __len__(self):
#         length = len(self.all_data)
#         return length
        

In [17]:
print("~~")

~~


In [18]:
train_transforms = transforms.Compose([
        utils.PointSampler(1024),
        utils.Normalize(),
        utils.RandRotation_z(),
        utils.RandomNoise(),
        utils.ToTensor()
    ])
valid_transforms = transforms.Compose([
        utils.PointSampler(1024),
        utils.Normalize(),
        utils.ToTensor()
    ])

In [19]:
root_path = 'C:/Users/dhkim/Desktop/directory/k-ium/data/'
batch_size =32

### Pytorch 데이터 클래스 생성
train_class = Custom_dataset(root_path=root_path, mode='train_3d', transforms=train_transforms)
valid_class = Custom_dataset(root_path=root_path, mode='valid_3d', transforms=valid_transforms)

if __name__ == '__main__':
    train_loader = DataLoader(train_class, batch_size=batch_size, shuffle = True, num_workers=12)
    valid_loader = DataLoader(valid_class, batch_size=batch_size, shuffle = False, num_workers=12)

In [20]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)


In [21]:
lr = 0.0001
# batch_size =32
epochs = 50

    
pointnet = model.PointNet()
pointnet.to(device)
optimizer = torch.optim.Adam(pointnet.parameters(), lr=lr)

In [22]:
print('Start training')
for epoch in tqdm(range(epochs)):
    pointnet.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device).float()
        labels = labels.to(device)

#             print(inputs)
#             print(labels)
        optimizer.zero_grad()
        outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))
#             print(outputs)
#             print(m3x3)
#             print(m64x64)
#             print(labels)

        loss = pointnetloss(outputs, labels, m3x3, m64x64)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
                print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                    (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                running_loss = 0.0

    pointnet.eval()
    correct = total = 0

    # validation
    if valid_loader:
        with torch.no_grad():
            for (inputs, labels) in valid_loader:
                inputs = inputs.to(device).float()
                labels = labels.to(device)
                outputs, __, __ = pointnet(inputs.transpose(1,2))
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_acc = 100. * correct / total
        print('Valid accuracy: %d %%' % val_acc)
    # save the model

#     checkpoint = Path(args.save_model_path)/'save_'+str(epoch)+'.pth'
#     torch.save(pointnet.state_dict(), checkpoint)
#     print('Model saved to ', checkpoint)

Start training


  0%|                                                                                           | 0/50 [00:46<?, ?it/s]


KeyboardInterrupt: 